In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141458
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-141458


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_name = 'cpu-cluster'

try: 
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except:
    config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0.1, 1.0)})

# Specify a Policy
policy = BanditPolicy(slack_amount=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', compute_target=compute_target, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config= HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=5, policy=policy)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=hyperdrive_run.id, filename='./outputs/best_hyperdrive_model.joblib')

['./outputs/best_hyperdrive_model.joblib']

In [8]:
joblib.load('./outputs/best_hyperdrive_model.joblib')

'HD_0d6c7342-58d5-4352-aa43-5977d7165272'

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['label'] = y

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x,
    label_column_name='label',
    n_cross_validations=3)

In [12]:
# Submit your automl run

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "Banking")
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_2d1b8515-82a1-4563-8791-fdd93521d4bd

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [14]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()
best_run.register_model(model_name = 'autml_best.pkl', model_path='./outputs')

Model(workspace=Workspace.create(name='quick-starts-ws-141458', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-141458'), name=autml_best.pkl, id=autml_best.pkl:1, version=1, tags={}, properties={})

In [15]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=1.9791666666666667,
                                                                                               scale_pos_weight=1,
                                                                                               subsample=0.6,
                                         